<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Impronte/Addestramento_scanner_Pytorch(Test_media)_255.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

## Import librerie

In [ ]:
#Librerie

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import shutil
import copy


##Check device


In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

1.9.0+cu111
True
Tesla P100-PCIE-16GB
cuda:0


##Def path

In [ ]:
#scanner
scanner_name = 'HiScan'

#salvataggio modello
path_model_save = F"/content/gdrive/My Drive/ModelliCNN/Scanner/Final/{scanner_name}_255"

#dataset
pathDataset=F'/content/gdrive/MyDrive/Dataset_impronte/training/{scanner_name}_patch'
pathTestset=F'/content/gdrive/MyDrive/Dataset_impronte/test/{scanner_name}'

##Collegamento google drive

In [ ]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#**Dataset**

##Caricamento dataset e creazione Dataloader

In [ ]:
import torch
import torchvision
from torchvision import transforms

data_transform=transforms.Compose([transforms.Resize([224,224]),
        transforms.ToTensor(),
        lambda x: (x*255).type(torch.uint8)
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])


train_dataset = datasets.ImageFolder(pathDataset,transform=data_transform)

In [ ]:
print(len(train_dataset))

20000


##Nomi classi

In [ ]:
classes_name=train_dataset.classes
class_number=len(classes_name)
print(classes_name)
print(class_number)

['Live', 'Spoof']
2


# **Creazione modello**

##Load model pre-trained

In [ ]:
#model_conv = models.densenet201(pretrained=True,progress=True)
model_conv = models.vgg19(pretrained=True,progress=True)
for param in model_conv.parameters():
  param.requires_grad = False #non modifico i parametri della parte riguardante le features durante il training, poiché voglio usare quella parte così com'è stata pre-addestrata

print(model_conv)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

##Aggiunta classificatore e freeze parametri

In [ ]:
model_conv.classifier[6]=nn.Linear(4096,2)
#model_conv.classifier=nn.Linear(1920,2)

for param in model_conv.classifier.parameters():
  param.requires_grad = True #invece pongo il calcolo del gradiente del classificatore, ovvero attivo il calcolo dei pesi

print(model_conv) 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

#**Training**

##Def train definitiva

In [ ]:
from collections import OrderedDict
from functools import partial
import sys

import numpy as np
import sys
import torch

from functools import partial
from torch.utils.data import DataLoader, TensorDataset


##### Data utils #####

def log_to_message(log):
    fmt = "{0}: {1}"
    return "    ".join(fmt.format(k, v) for k, v in log.items())


class ProgressBar(object):
    """Cheers @ajratner"""

    def __init__(self, n, length=40):
        # Protect against division by zero
        self.n      = max(1, n)
        self.nf     = float(n)
        self.length = length
        # Precalculate the i values that should trigger a write operation
        #self.ticks = set([round(i/100.0 * n) for i in range(101)])
        #self.ticks.add(n-1)
        self.ticks=range(n)
        self.bar(0)

    def bar(self, i, message=""):
        """Assumes i ranges through [0, n-1]"""
        if i in self.ticks:
            b = int(np.ceil(((i+1) / self.nf) * self.length))
            sys.stdout.write("\r[{0}{1}] {2}%\t{3}".format(
                "="*b, " "*(self.length-b), int(100*((i+1) / self.nf)), message
            ))
            sys.stdout.flush()

    def close(self, message=""):
        # Move the bar to 100% before closing
        self.bar(self.n-1)
        sys.stdout.write("{0}\n\n".format(message))
        sys.stdout.flush()



def training_f(train_dataset, numEpochs, model_conv, criterionCNN, optimizer_conv,batch_size, validation_split=0):
  
  best_acc = 0
  best_loss=0
  best_epoca = 0
  
  best_model_wts = copy.deepcopy(model_conv.state_dict())

  if validation_split:
      train_size = int(len(train_dataset) * 0.8)
      val_size = len(train_dataset) - train_size
      train_dataset, val_set = torch.utils.data.random_split(train_dataset, [train_size, val_size])
      traingen = torch.utils.data.DataLoader(train_dataset, pin_memory=True, batch_size=batch_size, shuffle=True,num_workers=4)
      valgen = torch.utils.data.DataLoader(val_set, pin_memory=True, batch_size=batch_size)
  
  else: traingen = torch.utils.data.DataLoader(train_dataset, pin_memory=True, batch_size=batch_size, shuffle=True,num_workers=4)
      
  for epochs in range(1,numEpochs + 1):
    
    print("Epoch {0} / {1}".format(epochs, numEpochs))
    log = OrderedDict()
    modelLoss_train = 0.0
    modelAcc_train = 0.0
    
    model_conv.train() 
    
    totalSize = 0
    batch_i=0
    num_batch=int(np.ceil(len(train_dataset)/batch_size))
    pb = ProgressBar(num_batch)
    #for each batch: operazioni per gli algoritmi di ottimizzazione
    for inputs,labels in traingen:
      inputs = inputs.type(torch.FloatTensor).cuda()
      labels = labels.cuda()
      
      batch_i=batch_i+1
      
      optimizer_conv.zero_grad()
      model_conv.zero_grad()
      
      y = model_conv(inputs)
      outp, preds = torch.max(y, 1)   
        
      lossCNN = criterionCNN(y, labels) #media per batch

      modelLoss_train += lossCNN.item() * inputs.size(0)
      totalSize += inputs.size(0)
      modelAcc_train += torch.sum(preds == labels.data).item()

      log['batch']="{0} / {1}".format(batch_i, num_batch)
      log['loss'] =round(modelLoss_train/totalSize,4)
      log['acc']=round(modelAcc_train/totalSize,4)
      log['Validation']=False
      pb.bar(batch_i, log_to_message(log))

      lossCNN.backward()  # pred = f(x)   -> loss = L(f(x), l_true)
      
      optimizer_conv.step()
    
    #calcolo loss e accuracy
    modelLoss_epoch_train = modelLoss_train/totalSize
    modelAcc_epoch_train  = modelAcc_train/totalSize
    
    log['loss']=round(modelLoss_epoch_train,2)
    log['acc']=round(modelAcc_epoch_train*100,2)
    pb.bar(num_batch-1, log_to_message(log))

    if validation_split:    
        #validation
               
        model_conv.eval()
        totalSize_val = 0
        modelLoss_val = 0.0
        modelAcc_val = 0.0

        log['Validation']=True
        log['loss_val']=modelLoss_val
        log['acc_val']=modelAcc_val
        pb.bar(num_batch-1, log_to_message(log))
        batch_val_i=0
        size_valgen=len(valgen)
        for inputs,labels in valgen:
          inputs = inputs.type(torch.FloatTensor).cuda()
          labels = labels.cuda()
          batch_val_i+=1
          y = model_conv(inputs)
          outp, preds = torch.max(y, 1)
          lossCNN = criterionCNN(y, labels)

          modelLoss_val += lossCNN.item() * inputs.size(0)
          totalSize_val += inputs.size(0)
          modelAcc_val += torch.sum(preds == labels.data).item()        

          log['batch_val']="{0} / {1}".format(batch_val_i,size_valgen )
          log['loss_val'] =round(modelLoss_val/totalSize_val,4)
          log['acc_val']=round(modelAcc_val/totalSize_val,4)
          pb.bar(num_batch-1, log_to_message(log))

        modelLoss_epoch_val=modelLoss_val/totalSize_val
        modelAcc_epoch_val = modelAcc_val/totalSize_val

        log['loss_val'] =round(modelLoss_epoch_val,2)
        log['acc_val']=round(modelAcc_epoch_val*100,2)
        
        if (modelAcc_epoch_val > best_acc) or (modelAcc_epoch_val == best_acc and modelLoss_epoch_val < best_loss) :
          best_acc = modelAcc_epoch_val
          best_loss = modelLoss_epoch_val
          best_epoca = epochs
          best_model_wts = copy.deepcopy(model_conv.state_dict()) #salvo come modello quello che mi ha restituito risultati migliori in validation
          
        pb.close(log_to_message(log))

    else: 
      best_model_wts = copy.deepcopy(model_conv.state_dict())
      pb.close(log_to_message(log))


  #pb.close(log_to_message(log))

  model_conv.load_state_dict(best_model_wts)
  return model_conv




##Set iperparametri

In [ ]:
#iper-parametri iniziali
learning_rate = 1e-6
num_epoch = 20
batch_size = 128
momentum=0.9
#richiamo funzione che effettua il training con validation

model_conv = model_conv.cuda() #sposta i calcoli sulla gpu
criterion = nn.CrossEntropyLoss() #criterio dell'aggiornamento del gradiente: minimizzazione funzione loss entropia
optimizer_conv = optim.SGD(model_conv.classifier.parameters(),lr=learning_rate,momentum=momentum)

##Addestramento

In [ ]:
best_model=training_f(train_dataset, num_epoch, model_conv, criterion, optimizer_conv,batch_size=batch_size, validation_split=0.2)

Epoch 1 / 20
[=                                       ] 0%	

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[========================================] 100%	batch: 125 / 125    loss: 3.0    acc: 59.03    Validation: True    loss_val: 0.71    acc_val: 77.48    batch_val: 32 / 32

Epoch 2 / 20
[========================================] 100%	batch: 125 / 125    loss: 1.81    acc: 69.27    Validation: True    loss_val: 0.58    acc_val: 81.25    batch_val: 32 / 32

Epoch 3 / 20
[========================================] 100%	batch: 125 / 125    loss: 1.51    acc: 72.47    Validation: True    loss_val: 0.49    acc_val: 83.25    batch_val: 32 / 32

Epoch 4 / 20
[========================================] 100%	batch: 125 / 125    loss: 1.3    acc: 74.39    Validation: True    loss_val: 0.45    acc_val: 84.5    batch_val: 32 / 32

Epoch 5 / 20
[========================================] 100%	batch: 125 / 125    loss: 1.18    acc: 75.81    Validation: True    loss_val: 0.42    acc_val: 85.02    batch_val: 32 / 32

Epoch 6 / 20
[========================================] 100%	batch: 125 / 125    loss: 1.12

#**Salvataggio modello**

In [ ]:
#salva modello su drive
torch.save(best_model.state_dict(),path_model_save)

#**Testing**

##test2 (tensori)

###Caricamento dataset

In [ ]:
data_transform_test=transforms.Compose([transforms.ToTensor(),
        lambda x: (x*255).type(torch.uint8)
        ])

test_dataset = datasets.ImageFolder(pathTestset,transform=data_transform_test)
print(len(test_dataset))

2500


###Dataloader

In [ ]:
testgen=torch.utils.data.DataLoader(test_dataset, pin_memory=True, batch_size=1,num_workers=4)
print(len(testgen))

2500


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


###calcolo dim

In [ ]:
def calc_size(n):
  return tuple(int(np.ceil(i * (80/100))) for i in n)

###test

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#best_model=model_conv
Test = pd.DataFrame()
best_model.eval()
best_model.cuda()

prob = nn.Softmax()

data_transform_test= transforms.Compose([transforms.Resize([224,224]),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
i=0
for input,label in testgen:
  i+=1
  sys.stdout.write("\rImage {0}/{1}".format(i,len(testgen)))
  sys.stdout.flush()
  n=input.shape
  n_mod=calc_size(n[2:4])
  crop_transform=transforms.TenCrop((n_mod[0],n_mod[1])).to(device)
  crops=crop_transform(input)
  live=0
  spoof=0
  for crop in crops:
    crop=data_transform_test(crop).to(device)
    crop=crop.type(torch.FloatTensor).cuda()
    outputs = best_model(crop)
    live+=outputs[0][0]
    spoof+=outputs[0][1]
  live=live/10
  spoof=spoof/10
  outputs=torch.Tensor([live,spoof])
  outputs=prob(outputs)
  predicted=np.argmax(outputs)
  Test = Test.append({'real': classes_name[int(label)] ,
                        'predicted': classes_name[predicted],
                        'value_pred_live': outputs[0].numpy(),
                        'value_pred_spoof': outputs[1].numpy()},ignore_index = True)  

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Image 3/2500

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Image 2500/2500

###accuracy

In [ ]:
true_label = Test.real.values
predicted = Test.predicted.values

print(round((np.sum((true_label == predicted).astype(int)))/Test.shape[0],4)*100)
#88

86.83999999999999


In [ ]:
print(Test.shape)

(2500, 4)


#Save pd

In [ ]:

Test.to_excel(scanner_name+'2.xlsx',index=False)
!cp '{scanner_name}2.xlsx' '/content/gdrive/MyDrive/Dataset_impronte/test/Preds_value/'